# 3. Tensorboard-可视化工具包
原Tensorflow的最强可视化工具Tensorboard已在Torch 1.4+版本进行了适配，现在可以搭配`torch.utils.tensorboard.SummaryWriter`进行使用。

Tensorboard支持追踪和可视化量化指标（如loss和accuracy），可视化模型图结构（没有Netron直观好用），展示直方图、图片等等。

这里贴个官方文档[简介](https://pytorch.org/tutorials/recipes/recipes/tensorboard_with_pytorch.html)与[详细版](https://pytorch.org/tutorials/intermediate/tensorboard_tutorial.html)，关于[tensorboard的文档](https://pytorch.org/docs/stable/tensorboard.html)

## 3.1 结合SummaryWriter使用Tensorboard简洁教程

### 3.1.1 SummaryWriter保存标量Log信息
利用`SummaryWriter`的`add_scalar(tag, scalar_value, global_step=None, walltime=None)`方法Log标量信息，默认保存在`./runs/`目录下。

下面演示一下线性回归训练过程的标量Log信息保存

In [5]:
import torch
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter('./data/log/',comment='LinearRegression')

# Data
x = torch.arange(-5, 5, 0.1).view(-1, 1) # -5:0.1:5
y = -5 * x + 0.1 * torch.randn(x.size()) # y = -5x + \epsilon
# Linear Regression
model = torch.nn.Linear(1, 1)
criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 1e-2)

# Train
def LR_training(iter):
    for epoch in range(iter):
        y1 = model(x)
        loss = criterion(y1, y)
        writer.add_scalar("Loss/train", loss, epoch) # global_step indicates the horizotal coordinate
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

if __name__ == "__main__":
    epoch = 100
    LR_training(epoch)
    writer.flush() # make sure all the pending events have been written to disk
    writer.close() # if u don't want to summary anything

### 3.1.2 使用Tensorboard查看Log信息
而后在终端输入`tensorboard --logdir=data/log`，其中logdir为保存log的路径，然后浏览器打开 http://localhost:6006/ 即可看到Log中的标量信息：

<p align=center>
<img src="./fig/3-1.png" width=1000>
</p>

其中smoothing对结果做了平滑，用来看loss下降的趋势，默认为0.6

## 3.2 SummaryWriter文档总结
具体调用方式可以参考[torch.utils.tensorboard](https://pytorch.org/docs/stable/tensorboard.html)

现假设利用SummaryWriter实例化了一个对象`writer`，可以使用`writer.add_scalar()`等方法向Log日志中添加输出信息，支持以下方法：

|方法名|作用|
|---|---|
|add_scalar|1个tag加入1个标量|
|add_scalars|1个tag加入多个标量|1=
|add_histogram|加入直方图|
|add_image|1个tag加入1张图片(3,H,W)|
|add_images|1个tag加入多张图片(N,3,H,W)|
|add_figure|将matplotlib的图加入|
|add_text|加入语言描述，没啥用|
|add_graph|加入模型结构图，我的评价是不如Netron|
|add_embedding|加入低维embedding用于可视化|
|add_mesh|加入网格/3D点云并可视化|
|add_hparams|加入一系列超参来比对效果，很实用|

## 3.3 Tensorboard常用方法与示意图
以1-TorchModel.ipynb中VAE模型处理MNIST数据为例，阐述Tensorboard的常用方法及其示意图。

In [3]:
from lib import TorchModel

for epoch in range(1, args.epochs +1):
    train(epoch)
    test(epoch)
    # Test Decoder
    with torch.no_grad():
        sample = torch.randn(64, 20).to(args.device)
        sample = model.decode(sample).cpu()
        save_image(sample.view(64, 1, 28, 28),
            './data/results/sample_'+str(epoch)+'.png')      


RuntimeError: Dataset not found. You can use download=True to download it